In [1]:
import os
os.chdir("../")

In [2]:
from storage.har_datasets import STSDataset, StreamingTimeSeries, StreamingTimeSeriesCopy
from s3ts.dtw_layer.lightningstsds import LSTSDataset
from s3ts.dtw_layer.dtw_layer import DTWLayer
from s3ts.dtw_layer.wrapper import lmodel
from pytorch_lightning import Trainer

In [3]:
from s3ts.api.ucr import load_ucr_classification
from s3ts.api.ts2sts import finite_random_STS
import numpy as np
import torch

X, Y, mapping = load_ucr_classification("BasicMotions")
#X, Y, mapping = load_ucr_classification("GunPoint")
print(X.shape, Y.shape, len(np.unique(Y)))

STS, SCS = finite_random_STS(X, Y, length=60)
print(STS.shape, SCS.shape)

Loading 'BasicMotions' from cache...
(80, 6, 100) (80,) 4
(6, 6000) (6000,)


In [4]:
ds = StreamingTimeSeries(STS, SCS, wsize=32, wstride=1)

In [5]:
indices_shuffled = np.arange(ds.indices.shape[0])[:5000]
np.random.shuffle(indices_shuffled)

data_split = {
    "train": lambda x: np.isin(x, indices_shuffled),
    "val": lambda x: np.isin(x, np.arange(ds.indices.shape[0])[5064:5500]),
    "test": lambda x: np.isin(x, np.arange(ds.indices.shape[0])[5064:]),
}

dm = LSTSDataset(ds, data_split=data_split, batch_size=32, random_seed=42, num_workers=16)

In [6]:
model = torch.nn.Sequential(
    DTWLayer(8, 6, 16, l_out=16, rho=0.01),
    torch.nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5),
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2),
    torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4),
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=128, out_features=4)
)

In [7]:
trainer = Trainer(max_epochs=50, accelerator="auto")
model_lightning = lmodel(model, 4, lr=0.001)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(model=model_lightning, datamodule=dm)


  | Name       | Type               | Params
--------------------------------------------------
0 | model      | Sequential         | 190 K 
1 | softmax    | Softmax            | 0     
2 | train_acc  | MulticlassAccuracy | 0     
3 | train_f1   | MulticlassF1Score  | 0     
4 | val_acc    | MulticlassAccuracy | 0     
5 | val_f1     | MulticlassF1Score  | 0     
6 | val_auroc  | MulticlassAUROC    | 0     
7 | test_acc   | MulticlassAccuracy | 0     
8 | test_f1    | MulticlassF1Score  | 0     
9 | test_auroc | MulticlassAUROC    | 0     
--------------------------------------------------
190 K     Trainable params
0         Non-trainable params
190 K     Total params
0.761     Total estimated model params size (MB)


Epoch 14:  65%|██████▍   | 101/156 [00:03<00:02, 25.27it/s, v_num=2, train_loss_step=0.366, val_loss=0.485, val_acc=0.878, val_auroc=0.128, train_loss_epoch=0.416, train_acc=0.876]